<a href="https://colab.research.google.com/github/AnIsAsPe/Estadistica_y_Probabilidad_para-CD-/blob/main/PrincipiosDeProbabilidad/Semana6/Metropolis_Hastings_para_decifrar_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Algoritmo Metropolis Hastings para decifrar textos <h2>

El presente código es una adaptación del presentado en [este repositorio](https://github.com/svivek/mcmc-decoding-example)

El ejemplo está inspirado en la historia descrita en [The Markov Chain Monte Carlo Revolution](https://www.ams.org/journals/bull/2009-46-02/S0273-0979-08-01238-X/S0273-0979-08-01238-X.pdf). 







<img src='https://pbs.twimg.com/media/DnJnUCpWwAAmPnR.jpg'>



El objetivo es decifrar los mensajes intercambiados por prisioneros, con la conjetura de que cada simbolo representa una letra o simbolo del lenguaje. La tarea es pues, encontrar ese diccionario o **clave** para codificar y decodificar los mensajes.

## Cifrado por sustitución

Es un método de cifrado para codificar y decodificar mensajes usando una clave que relaciona una letra o simbolo por otro.

Como ejemplo, se puede suponer que la clave de un cifrado está contenida en la siguiente tabla:

|original | codificado |
|---------|------------|
|a        | z          |
|n        | 4          |
|l        | 5          |
|i        | a          |
|s        | 8          |

De manera que la palabra  `analisis` puede ser codificada como `z4z5a8a8`. 

In [1]:
def code(text, key): 
    return ''.join(list(map(lambda char: key[char], text)))

In [2]:
clave = {'a': 'z', 'n': '4', 'l': '5', 'i': 'a', 's':'8'}
code('analisis', clave)

'z4z5a8a8'

Y podemos decodificar la palabra usando la clave inversa:

In [3]:
inv_clave = {v : k for k, v in clave.items()}
code('z4z5a8a8', inv_clave)

'analisis'

#¿Cómo encontrar una clave adecuada?

Si consideramos solamente el espacio en blanco, las letras en minúscula, y los números estamos hablando de 38 simbolos, y por tanto el espacio muestral en el que vive la clave es enorme.

In [4]:
alphabet = "abcdefghijklmnñopqrstuvwxyz0123456789 "

print('Total de simbolos considerados:', len(alphabet))

from math import factorial
print('Tamaño del espacio muestral en el que vive la clave que buscamos', factorial(38))

Total de simbolos considerados: 38
Tamaño del espacio muestral en el que vive la clave que buscamos 523022617466601111760007224100074291200000000


Supondremos que el lenguaje de los mensajes cifrados es el español, por tanto utilizaremos como guía las probabilidades de transición de los carácteres en español.

# Calificación de las claves candidatas

Para cada secuencia de caracteres $c_1c_2c_3\cdots c_n$, estimaresmos su probabilidad de la siguiente manera

$$P(c_1c_2c_3\cdots c_n) = P(c_1) P(c_2 \mid c_1) P(c_3 \mid c_2) \cdots P(c_n \mid c_{n-1})$$

Así, la probabilidad de la palabra `analisis` se calcularía como 

 $P(a)~P(n \mid a)P(a \mid n)P(l \mid a)P(i \mid l)P(s \mid i)P(i \mid s)P(s \mid i)$.

Para hacer lo anterior necesitamos una matriz de transición de un caracter a otro, y en este sentido, para facilitar el análisis,  ignoraremos el primer caracter porque no hará practicamente diferencia en una secuencia larga.

## ¿Cómo obtener la matriz de transición de los caracteres? 


In [5]:
# ¿Cuántas probabilidades de transición debe tener nuestra matriz de transición?
len(alphabet)**2

1444

Para aprender estas probabilidades usaremos un texto, lo más largo posible en español

In [6]:
import re

def limpiar_texto(texto):

    #cambiar texto a minusculas
    texto = texto.lower()
    
    # quitar tildes
    a,b = 'áéíóúü','aeiouu'
    trans = str.maketrans( a,b)
    texto = texto.translate(trans)

    # solo letras y numeros
    texto = re.sub('[^ña-z0-9 ]+' ,' ', texto).strip()  

    # remplazar multiples espacios contiguos por un solo espacio
    texto = re.sub('\s+',' ', texto)                    
    return texto

def find_between( s, first, last ):
    '''fuente: https://stackoverflow.com/questions/3368969/find-string-between-two-substrings'''
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
def leer_texto(file, inicio=None, fin=None):
    with open(file, 'r') as f:
        if inicio is not None and fin is not None:
            libro = find_between(f.read(), inicio, fin)
        else:
            libro = f.read()
    return limpiar_texto(libro)

Se puede calcular las probabilidades de transisión utilizando cualquier texto. Si este es suficientemente grande, podremos obtener buenas estimaciones. Usaremos Don Quijote de la Mancha, que tiene más de 2 millones de caracteres.

In [20]:
from collections import Counter
import math

def bigram_log_probabilities(book):
    
    character_counts = Counter(book)
    bigram_counts = Counter(zip(book, book[1:]))
    bigrams = bigram_counts.keys()
    return dict(map(lambda k: [k, math.log(bigram_counts[k]) - math.log(character_counts[k[0]])], bigrams))

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
inicio = "*** START OF THE PROJECT GUTENBERG EBOOK DON QUIJOTE ***"
fin = "*** END OF THE PROJECT GUTENBERG EBOOK DON QUIJOTE ***"

file = '/content/drive/MyDrive/Datos/DonQuijote.txt'
DonQ = leer_texto(file, inicio, fin )

In [21]:
bigram_table = bigram_log_probabilities(DonQ)
bigram_table

{('e', 'l'): -2.495635370118805,
 ('l', ' '): -1.633573260510028,
 (' ', 'i'): -4.937384512589171,
 ('i', 'n'): -2.3504318062150116,
 ('n', 'g'): -4.031173508631302,
 ('g', 'e'): -2.980174879019078,
 ('e', 'n'): -1.8827851415801167,
 ('n', 'i'): -3.196628842521836,
 ('i', 'o'): -2.349621480303913,
 ('o', 's'): -1.8348327937786753,
 ('s', 'o'): -2.8913442783423715,
 ('o', ' '): -0.8242763995237041,
 (' ', 'h'): -3.420071451532097,
 ('h', 'i'): -2.3957986266629003,
 ('i', 'd'): -2.801143855311272,
 ('d', 'a'): -2.023135023232147,
 ('a', 'l'): -2.6003410610565005,
 ('l', 'g'): -4.105093437001927,
 ('g', 'o'): -1.4025662217672572,
 (' ', 'd'): -2.166409555430894,
 ('d', 'o'): -1.3743360383401892,
 ('o', 'n'): -2.0723476075124374,
 ('n', ' '): -1.137340896369082,
 (' ', 'q'): -2.633056752874694,
 ('q', 'u'): 0.0,
 ('u', 'i'): -2.480561604574996,
 ('i', 'j'): -2.933481589751146,
 ('j', 'o'): -0.5809899206883387,
 ('o', 't'): -3.485054422488467,
 ('t', 'e'): -1.3776567950215597,
 ('e', ' '): 

In [11]:
len(bigram_table)

409

In [54]:
bigram_table[b]

KeyError: ignored

Y mediante la siguiente función calcularemos que tan "buena" es la secuencia utilizando para ello las probabilidades de transición estimadas.

In [22]:
LOG_EPSILON = -30  # probabilidad muy pequeña vista como logaritmo, que se ustilizara cuando no exista probabilidad de trancisión de una letra a otra

def goodness(text, bigram_table):
    bigrams = zip(text, text[1:])
    sum = 0
    for b in bigrams:
        sum += bigram_table.get(b) or LOG_EPSILON
    return sum

In [23]:
print("Log probability of 'opamc' = " +str(goodness("opamc", bigram_table)))

Log probability of 'opamc' = -40.784560457172915


In [24]:
print("Log probabilidad de 'campo' = " +str(goodness("campo", bigram_table)))

Log probabilidad de 'campo' = -9.40201613643682


# Implementación

## Creación del mensaje encriptado

In [25]:
plain_text = limpiar_texto(
    """Muchos años después, frente al pelotón de fusilamiento, el coronel Aureliano Buendía había de 
recordar aquella tarde remota en que su padre lo llevó a conocer el hielo. Macondo era entonces 
una aldea de veinte casas de barro y cañabrava construidas a la orilla de un río de aguas diáfanas 
que se precipitaban por un lecho de piedras pulidas, blancas y enormes como huevos 
prehistóricos. El mundo era tan reciente, que muchas cosas carecían de nombre, y para 
mencionarlas había que señalarías con el dedo. Todos los años, por el mes de marzo, una familia 
de gitanos desarrapados plantaba su carpa cerca de la aldea, y con un grande alboroto de pitos y 
timbales daban a conocer los nuevos inventos. Primero llevaron el imán. Un gitano corpulento, de 
barba montaraz y manos de gorrión, que se presentó con el nombre de Melquíades, hizo una 
truculenta demostración pública de lo que él mismo llamaba la octava maravilla de los sabios 
alquimistas de Macedonia. Fue de casa en casa arrastrando dos lingotes metálicos, y todo el 
mundo se espantó al ver que los calderos, las pailas, las tenazas y los anafes se caían de su sitio, 
y las maderas crujían por la desesperación de los clavos y los tornillos tratando de desenclavarse, 
y aun los objetos perdidos desde hacía mucho tiempo aparecían por donde más se les había 
buscado, y se arrastraban en desbandada turbulenta detrás de los fierros mágicos de Melquíades. 
"""
)

In [26]:
import random

def make_random_key():
    l = list(alphabet)
    random.shuffle(l)
    return dict(zip(alphabet, l))

In [27]:
key = make_random_key()
cipher_text = code(plain_text, key)
cipher_text

'3rg4jmqx6jmq5ym0rymqcaytdyqxkq0ykjdjtq5yqcrmwkx3wytdjqykqgjajtykqxraykwxtjqñryt5wxq4xñwxq5yqaygja5xaqxzrykkxqdxa5yqay3jdxqytqzryqmrq0x5ayqkjqkky9jqxqgjtjgyaqykq4wykjq3xgjt5jqyaxqytdjtgymqrtxqxk5yxq5yq9ywtdyqgxmxmq5yqñxaajqvqgx6xñax9xqgjtmdarw5xmqxqkxqjawkkxq5yqrtqawjq5yqx8rxmq5wxcxtxmqzryqmyq0aygw0wdxñxtq0jaqrtqkyg4jq5yq0wy5axmq0rkw5xmqñkxtgxmqvqytja3ymqgj3jq4ry9jmq0ay4wmdjawgjmqykq3rt5jqyaxqdxtqaygwytdyqzryq3rg4xmqgjmxmqgxaygwxtq5yqtj3ñayqvq0xaxq3ytgwjtxakxmq4xñwxqzryqmy6xkxawxmqgjtqykq5y5jqdj5jmqkjmqx6jmq0jaqykq3ymq5yq3xaojqrtxqcx3wkwxq5yq8wdxtjmq5ymxaax0x5jmq0kxtdxñxqmrqgxa0xqgyagxq5yqkxqxk5yxqvqgjtqrtq8axt5yqxkñjajdjq5yq0wdjmqvqdw3ñxkymq5xñxtqxqgjtjgyaqkjmqtry9jmqwt9ytdjmq0aw3yajqkky9xajtqykqw3xtqrtq8wdxtjqgja0rkytdjq5yqñxañxq3jtdxaxoqvq3xtjmq5yq8jaawjtqzryqmyq0aymytdjqgjtqykqtj3ñayq5yq3ykzrwx5ymq4wojqrtxqdargrkytdxq5y3jmdaxgwjtq0rñkwgxq5yqkjqzryqykq3wm3jqkkx3xñxqkxqjgdx9xq3xax9wkkxq5yqkjmqmxñwjmqxkzrw3wmdxmq5yq3xgy5jtwxqcryq5yqgxmxqytqgxmxqxaaxmdaxt5jq5jmqkwt8jdymq3ydxkwgjmqvqdj5

## Busqueda de la clave

Empezaremos la busqueda de la clave adecuado, utilizando en el inicio una **clave** aleatoria que iremos cambiando por una  clave cercana, simplemente transponiendo dos caracteres seleccionados aleatoriamente.

In [30]:
clave0 = make_random_key()  # creamos una clave inicial
clave0

{'a': 'w',
 'b': 'h',
 'c': '5',
 'd': 'l',
 'e': '2',
 'f': 'v',
 'g': 'x',
 'h': 'i',
 'i': 'c',
 'j': 'p',
 'k': 's',
 'l': '4',
 'm': '8',
 'n': 'z',
 'ñ': '1',
 'o': 'o',
 'p': 't',
 'q': '9',
 'r': 'y',
 's': 'q',
 't': '3',
 'u': 'k',
 'v': 'm',
 'w': ' ',
 'x': '7',
 'y': 'n',
 'z': 'e',
 '0': 'a',
 '1': 'u',
 '2': 'ñ',
 '3': '6',
 '4': 'f',
 '5': 'r',
 '6': 'b',
 '7': '0',
 '8': 'g',
 '9': 'd',
 ' ': 'j'}

In [31]:
decoded0 =code(cipher_text, clave0) # decodificamos
decoded0

'6yxfp897bp89rn8ayn895wn3ln97s9ansplp39rn95y8 s76 n3lp9ns9xpwp3ns97ywns 73p91yn3r 79f71 79rn9wnxpwr7w97eynss79l7wrn9wn6pl79n39eyn98y9a7rwn9sp9ssndp979xp3pxnw9ns9f nsp967xp3rp9nw79n3lp3xn89y3797srn79rn9dn 3ln9x78789rn917wwp9m9x7b71w7d79xp38lwy r78979s79pw ss79rn9y39w p9rn97gy789r 7573789eyn98n9awnx a l71739apw9y39snxfp9rn9a nrw789ays r7891s73x789m9n3pw6n89xp6p9fyndp89awnf 8lpw xp89ns96y3rp9nw79l739wnx n3ln9eyn96yxf789xp8789x7wnx 739rn93p61wn9m9a7w796n3x p37ws789f71 79eyn98nb7s7w 789xp39ns9rnrp9lprp89sp897bp89apw9ns96n89rn967wop9y379576 s 79rn9g l73p89rn87ww7a7rp89as73l71798y9x7wa79xnwx79rn9s797srn79m9xp39y39gw73rn97s1pwplp9rn9a lp89m9l 617sn89r7173979xp3pxnw9sp893yndp89 3dn3lp89aw 6nwp9ssnd7wp39ns9 6739y39g l73p9xpwaysn3lp9rn917w1796p3l7w7o9m9673p89rn9gpww p39eyn98n9awn8n3lp9xp39ns93p61wn9rn96nsey 7rn89f op9y379lwyxysn3l79rn6p8lw7x p39ay1s x79rn9sp9eyn9ns96 86p9ss767179s79pxl7d7967w7d ss79rn9sp89871 p897sey 6 8l789rn967xnrp3 795yn9rn9x7879n39x78797ww78lw73rp9rp89s 3gpln896nl7s xp89m9lpr

In [32]:
log_p0 = goodness(decoded0, bigram_table)  # evaluamos
log_p0

-34440.49418719469

In [33]:
pair = random.sample(alphabet, 2)
pair[0], pair[1]

('e', '4')

In [34]:
clave0[pair[0]] ,clave0[pair[1]]

('2', 'f')

In [35]:
clave1 = clave0.copy()
clave1[pair[0]] = clave0[pair[1]]
clave1[pair[1]] = clave0[pair[0]]
clave1

{'a': 'w',
 'b': 'h',
 'c': '5',
 'd': 'l',
 'e': 'f',
 'f': 'v',
 'g': 'x',
 'h': 'i',
 'i': 'c',
 'j': 'p',
 'k': 's',
 'l': '4',
 'm': '8',
 'n': 'z',
 'ñ': '1',
 'o': 'o',
 'p': 't',
 'q': '9',
 'r': 'y',
 's': 'q',
 't': '3',
 'u': 'k',
 'v': 'm',
 'w': ' ',
 'x': '7',
 'y': 'n',
 'z': 'e',
 '0': 'a',
 '1': 'u',
 '2': 'ñ',
 '3': '6',
 '4': '2',
 '5': 'r',
 '6': 'b',
 '7': '0',
 '8': 'g',
 '9': 'd',
 ' ': 'j'}

In [36]:
decoded1 =code(cipher_text, clave1) # decodificamos
decoded1

'6yx2p897bp89rn8ayn895wn3ln97s9ansplp39rn95y8 s76 n3lp9ns9xpwp3ns97ywns 73p91yn3r 79271 79rn9wnxpwr7w97eynss79l7wrn9wn6pl79n39eyn98y9a7rwn9sp9ssndp979xp3pxnw9ns92 nsp967xp3rp9nw79n3lp3xn89y3797srn79rn9dn 3ln9x78789rn917wwp9m9x7b71w7d79xp38lwy r78979s79pw ss79rn9y39w p9rn97gy789r 7573789eyn98n9awnx a l71739apw9y39snx2p9rn9a nrw789ays r7891s73x789m9n3pw6n89xp6p92yndp89awn2 8lpw xp89ns96y3rp9nw79l739wnx n3ln9eyn96yx2789xp8789x7wnx 739rn93p61wn9m9a7w796n3x p37ws789271 79eyn98nb7s7w 789xp39ns9rnrp9lprp89sp897bp89apw9ns96n89rn967wop9y379576 s 79rn9g l73p89rn87ww7a7rp89as73l71798y9x7wa79xnwx79rn9s797srn79m9xp39y39gw73rn97s1pwplp9rn9a lp89m9l 617sn89r7173979xp3pxnw9sp893yndp89 3dn3lp89aw 6nwp9ssnd7wp39ns9 6739y39g l73p9xpwaysn3lp9rn917w1796p3l7w7o9m9673p89rn9gpww p39eyn98n9awn8n3lp9xp39ns93p61wn9rn96nsey 7rn892 op9y379lwyxysn3l79rn6p8lw7x p39ay1s x79rn9sp9eyn9ns96 86p9ss767179s79pxl7d7967w7d ss79rn9sp89871 p897sey 6 8l789rn967xnrp3 795yn9rn9x7879n39x78797ww78lw73rp9rp89s 3gpln896nl7s xp89m9lpr

In [37]:
log_p1 = goodness(decoded1, bigram_table)  # evaluamos
log_p1

-34530.50624395321

En base a lo anterior ¿Aceptamos la nueva clave propuesta?

## Algoritmo completo

In [38]:
def transpose_random(key):
    pair = random.sample(alphabet, 2)
    new_key = dict(key)
    new_key[pair[0]] = key[pair[1]]
    new_key[pair[1]] = key[pair[0]]
    return new_key

def decode(cipher_text, bigram_table, iters = 10000, print_every = 1000):
    # 1. inicializamos con una clave aleatoria
    current_key = make_random_key()  

    for i in range(0, iters):  # en cada iteración utilizaremos una nueva clave

        # 2. decodificamos la secuencia con la clave
        decoded = code(cipher_text, current_key) 

        if i % print_every == 0:
            print(str(i) + "\t" + decoded + "\n")

        # 3. evaluamos la secuencia utilizando el modelo de bigramas 
        score = goodness(decoded, bigram_table)   
               
        # 4. proponemos una nueva clave "cercana"
        changed_key = transpose_random(current_key) 

        # 5. evaluamos la nueva clave decodificando la secuencia  
        changed_score = goodness(code(cipher_text, changed_key), bigram_table)  # evaluamos la secuencia con la nueva clave propuesta

        # 6.1 si la clave propuesta da mejores resultados nos quedamos con ella para la prox iteración
        if changed_score > score:   
            current_key = changed_key

        # damos oportunidad proporcional a la relación entre la probabilidad de las dos secuencias evaluadas 
        else:  
            diff = changed_score - score
            if math.log(random.random())  < diff:
                current_key = changed_key

    decoded = code(cipher_text, current_key)
    print("Final decoded: " + decoded)
    return decoded

In [39]:
decode(cipher_text, bigram_table, 10000)

0	mplywn20hwn2v5nzp5n2ab5 452082z58w4w 2v52apno80mo5 4w2582lwbw 5820pb58o0 w2sp5 vo02y0so02v52b5lwbv0b20tp5880240bv52b5mw4025 2tp52np2z0vb528w28856w202lw wl5b2582yo58w2m0lw vw25b025 4w l5n2p 0208v502v5265o 452l0n0n2v52s0bbw2g2l0h0sb0602lw n4bpov0n202802wbo8802v52p 2bow2v5203p0n2vo0a0 0n2tp52n52zb5lozo40s0 2zwb2p 285lyw2v52zo5vb0n2zp8ov0n2s80 l0n2g25 wbm5n2lwmw2yp56wn2zb5yon4wbolwn2582mp vw25b0240 2b5lo5 452tp52mply0n2lwn0n2l0b5lo0 2v52 wmsb52g2z0b02m5 low 0b80n2y0so02tp52n5h080bo0n2lw 2582v5vw24wvwn28wn20hwn2zwb2582m5n2v52m0buw2p 02a0mo8o02v523o40 wn2v5n0bb0z0vwn2z80 40s02np2l0bz02l5bl02v5280208v502g2lw 2p 23b0 v5208swbw4w2v52zo4wn2g24oms085n2v0s0 202lw wl5b28wn2 p56wn2o 65 4wn2zbom5bw288560bw 2582om0 2p 23o40 w2lwbzp85 4w2v52s0bs02mw 40b0u2g2m0 wn2v523wbbow 2tp52n52zb5n5 4w2lw 2582 wmsb52v52m58tpo0v5n2youw2p 024bplp85 402v5mwn4b0low 2zps8ol02v528w2tp52582monmw2880m0s02802wl40602m0b06o8802v528wn2n0sown208tpomon40n2v52m0l5vw o02ap52v52l0n025 2l0n020bb0n4b0 vw2vwn28o 3w45n2m5408olwn2g24w

'muchos años despues frente al peloton de fusilamiento el coronel aureliano buendia habia de recordar avuella tarde remota en vue su padre lo llejo a conocer el hielo macondo era entonces una aldea de jeinte casas de barro y cañabraja construidas a la orilla de un rio de aguas diafanas vue se precipitaban por un lecho de piedras pulidas blancas y enormes como huejos prehistoricos el mundo era tan reciente vue muchas cosas carecian de nombre y para mencionarlas habia vue señalarias con el dedo todos los años por el mes de marzo una familia de gitanos desarrapados plantaba su carpa cerca de la aldea y con un grande alboroto de pitos y timbales daban a conocer los nuejos injentos primero llejaron el iman un gitano corpulento de barba montaraz y manos de gorrion vue se presento con el nombre de melvuiades hizo una truculenta demostracion publica de lo vue el mismo llamaba la octaja marajilla de los sabios alvuimistas de macedonia fue de casa en casa arrastrando dos lingotes metalicos y tod

# Otros ejemplos

In [40]:
# El principito. Tomado de https://archive.org/stream/ElPrincipitoAntoineDeSaintExupery/El_principito-_Antoine_De_Saint_Exupery_djvu.txt

plain_text = limpiar_texto(
    """Tengo poderosas razones para creer que el planeta del cual venía el principito era 
el asteroide B 612. Este asteroide ha sido visto sólo una vez con el telescopio en 
1909, por un astrónomo turco. 
Este astrónomo hizo una gran demostración de su descubrimiento en un congreso 
Internacional de Astronomía. Pero nadie le creyó a causa de su manera de vestir. 
Las personas mayores son así. Felizmente para la reputación del asteroide B 612, 
un dictador turco impuso a su pueblo, bajo pena de muerte, el vestido a la 
europea. Entonces el astrónomo volvió a dar cuenta de su descubrimiento en 1920 
y como lucía un traje muy elegante, todo el mundo aceptó su demostración. 
"""
)
cipher_text = code(plain_text, key)
cipher_text

'dyt8jq0j5yajmxmqaxojtymq0xaxqgayyaqzryqykq0kxtydxq5ykqgrxkq9ytwxqykq0awtgw0wdjqyaxqykqxmdyajw5yqñq2ulqymdyqxmdyajw5yq4xqmw5jq9wmdjqmjkjqrtxq9yoqgjtqykqdykymgj0wjqytqun1nq0jaqrtqxmdajtj3jqdragjqymdyqxmdajtj3jq4wojqrtxq8axtq5y3jmdaxgwjtq5yqmrq5ymgrñaw3wytdjqytqrtqgjt8aymjqwtdyatxgwjtxkq5yqxmdajtj3wxq0yajqtx5wyqkyqgayvjqxqgxrmxq5yqmrq3xtyaxq5yq9ymdwaqkxmq0yamjtxmq3xvjaymqmjtqxmwqcykwo3ytdyq0xaxqkxqay0rdxgwjtq5ykqxmdyajw5yqñq2ulqrtq5wgdx5jaqdragjqw30rmjqxqmrq0ryñkjqñxsjq0ytxq5yq3ryadyqykq9ymdw5jqxqkxqyraj0yxqytdjtgymqykqxmdajtj3jq9jk9wjqxq5xaqgrytdxq5yqmrq5ymgrñaw3wytdjqytqunl1qvqgj3jqkrgwxqrtqdaxsyq3rvqyky8xtdyqdj5jqykq3rt5jqxgy0djqmrq5y3jmdaxgwjt'

In [43]:
decode(cipher_text, bigram_table, 20000)

0	2v0cuenurv4ukfke4f5u0vkenf4fe84vv4esgvevjenjf0v2fervje8gfjeov0wfevjen4w08wnw2uev4fevjefk2v4uwrvetex6pevk2vefk2v4uwrvezfekwrueowk2uekujueg0feov5e8u0evje2vjvk8unwuev0e6i7ienu4eg0efk24u0uaue2g48uevk2vefk24u0uauezw5ueg0fec4f0ervauk24f8wu0ervekgervk8gt4wawv02uev0eg0e8u0c4vkuew02v40f8wu0fjervefk24u0uawfenv4ue0frwvejve84vhuefe8fgkfervekgeaf0v4ferveovk2w4ejfkenv4ku0fkeafhu4vkeku0efkweqvjw5av02venf4fejfe4vng2f8wu0ervjefk2v4uwrvetex6peg0erw82fru4e2g48uewangkuefekgengvtjuetfduenv0ferveagv42vevjeovk2wruefejfevg4unvfev02u08vkevjefk24u0uaueoujowueferf4e8gv02fervekgervk8gt4wawv02uev0e6ip7ehe8uauejg8wfeg0e24fdveaghevjvcf02ve2uruevjeag0ruef8vn2uekgervauk24f8wu0

1000	tasfeibevaren nir xesanib r ilraariñoaiadibd sat ivadilo dijasu iadibruslubuteiar iadi ntareuvaicig60iantai ntareuvaiz inuveijunteinedeios ijaxilesiaditadanlebueiasi6151iberiosi ntresemeitorleiantai ntresemeizuxeios ifr sivamentr luesivainoivanlocrumuasteiasiosilesfraneiustars lues divai ntresemu ibareis vuaidailrapei il on ivainoim sar 

'tengo poderosas razones para creer fue el planeta del cual venia el principito era el asteroide b q10 este asteroide ha sido visto solo una vez con el telescopio en 16x6 por un astronomo turco este astronomo hizo una gran demostracion de su descubrimiento en un congreso internacional de astronomia pero nadie le creyo a causa de su manera de vestir las personas mayores son asi jelizmente para la reputacion del asteroide b q10 un dictador turco impuso a su pueblo baño pena de muerte el vestido a la europea entonces el astronomo volvio a dar cuenta de su descubrimiento en 160x y como lucia un trañe muy elegante todo el mundo acepto su demostracion'

In [44]:
# La liebre y la tortuga. Tomado de https://psicologiaymente.com/cultura/fabulas-de-esopo

plain_text = limpiar_texto(
    """Un día una liebre orgullosa y veloz, vió como una tortuga caminaba por el camino y se le acercó. La liebre empezó a burlarse de la lentitud del otro animal y de la longitud de sus patas. Sin embargo, la tortuga le respondió que estaba segura de que a pesar de la gran velocidad de la liebre era capaz de ganarla en una carrera.
La liebre, segura de su victoria y considerando el reto imposible de perder, aceptó. Ambos pidieron a la zorra que señalara la meta, a lo que esta aceptó, al igual que al cuervo para que hiciera de juez.
Al llegar el día de la competición, al empezar la carrera la liebre y la tortuga salieron al mismo tiempo. La tortuga avanzaba sin detenerse, pero lentamente.
La liebre era muy veloz, y viendo que sacaba una gran ventaja a la tortuga decidió ir parándose y descansando de vez en cuando. Pero en una de las ocasiones la liebre se quedó dormida. La tortuga, poco a poco, siguió avanzando.
Cuando la liebre despertó, se encontró con que la tortuga estaba a punto de cruzar la meta. Aunque echó a correr fue demasiado tarde y finalmente la tortuga ganó la carrera".
Esta fábula nos enseña que el trabajo duro, la perseverancia, la constancia y el esfuerzo nos llevarán a nuestras metas, aunque sea poco a poco, si no nos rendimos. También nos permite ver cómo la arrogancia, la falta de constancia y el exceso de seguridad en uno mismo nos puede llevar a perder oportunidades y a no alcanzar nuestras metas.
"""
)
cipher_text = code(plain_text, key)
cipher_text

'rtq5wxqrtxqkwyñayqja8rkkjmxqvq9ykjoq9wjqgj3jqrtxqdjadr8xqgx3wtxñxq0jaqykqgx3wtjqvqmyqkyqxgyagjqkxqkwyñayqy30yojqxqñrakxamyq5yqkxqkytdwdr5q5ykqjdajqxtw3xkqvq5yqkxqkjt8wdr5q5yqmrmq0xdxmqmwtqy3ñxa8jqkxqdjadr8xqkyqaym0jt5wjqzryqymdxñxqmy8raxq5yqzryqxq0ymxaq5yqkxq8axtq9ykjgw5x5q5yqkxqkwyñayqyaxqgx0xoq5yq8xtxakxqytqrtxqgxaayaxqkxqkwyñayqmy8raxq5yqmrq9wgdjawxqvqgjtmw5yaxt5jqykqaydjqw30jmwñkyq5yq0ya5yaqxgy0djqx3ñjmq0w5wyajtqxqkxqojaaxqzryqmy6xkxaxqkxq3ydxqxqkjqzryqymdxqxgy0djqxkqw8rxkqzryqxkqgrya9jq0xaxqzryq4wgwyaxq5yqsryoqxkqkky8xaqykq5wxq5yqkxqgj30ydwgwjtqxkqy30yoxaqkxqgxaayaxqkxqkwyñayqvqkxqdjadr8xqmxkwyajtqxkq3wm3jqdwy30jqkxqdjadr8xqx9xtoxñxqmwtq5ydytyamyq0yajqkytdx3ytdyqkxqkwyñayqyaxq3rvq9ykjoqvq9wyt5jqzryqmxgxñxqrtxq8axtq9ytdxsxqxqkxqdjadr8xq5ygw5wjqwaq0xaxt5jmyqvq5ymgxtmxt5jq5yq9yoqytqgrxt5jq0yajqytqrtxq5yqkxmqjgxmwjtymqkxqkwyñayqmyqzry5jq5ja3w5xqkxqdjadr8xq0jgjqxq0jgjqmw8rwjqx9xtoxt5jqgrxt5jqkxqkwyñayq5ym0yadjqmyqytgjtdajqgjtqzryqkxqdjadr8xqymdxñxqxq0rtdjq5yqgaroxaqkxq3ydxqxrtzryqyg4j

In [46]:
decode(cipher_text, bigram_table, 10000)

0	7wñzvañ7wañrv4px4ñ3xn7rr3oañjñu4r3iñuv3ñ03b3ñ7wañe3xe7nañ0abvwapañg3xñ4rñ0abvw3ñjño4ñr4ña04x03ñrañrv4px4ñ4bg4i3ñañp7xraxo4ñz4ñrañr4weve7zñz4rñ3ex3ñawvbarñjñz4ñrañr3wnve7zñz4ño7oñgaeaoñovwñ4bpaxn3ñrañe3xe7nañr4ñx4og3wzv3ñ274ñ4oeapaño4n7xañz4ñ274ñañg4oaxñz4ñrañnxawñu4r30vzazñz4ñrañrv4px4ñ4xañ0agaiñz4ñnawaxrañ4wñ7wañ0axx4xañrañrv4px4ño4n7xañz4ño7ñuv0e3xvañjñ03wovz4xawz3ñ4rñx4e3ñvbg3ovpr4ñz4ñg4xz4xña04ge3ñabp3oñgvzv4x3wñañrañi3xxañ274ño49araxañrañb4eañañr3ñ274ñ4oeaña04ge3ñarñvn7arñ274ñarñ074xu3ñgaxañ274ñ6v0v4xañz4ñy74iñarñrr4naxñ4rñzvañz4ñrañ03bg4ev0v3wñarñ4bg4iaxñrañ0axx4xañrañrv4px4ñjñrañe3xe7nañoarv4x3wñarñbvob3ñev4bg3ñrañe3xe7nañauawiapañovwñz4e4w4xo4ñg4x3ñr4weab4we4ñrañrv4px4ñ4xañb7jñu4r3iñjñuv4wz3ñ274ñoa0apañ7wañnxawñu4weayañañrañe3xe7nañz40vzv3ñvxñgaxawz3o4ñjñz4o0awoawz3ñz4ñu4iñ4wñ07awz3ñg4x3ñ4wñ7wañz4ñraoñ30aov3w4oñrañrv4px4ño4ñ274z3ñz3xbvzañrañe3xe7nañg303ñañg303ñovn7v3ñauawiawz3ñ07awz3ñrañrv4px4ñz4og4xe3ño4ñ4w03wex3ñ03wñ274ñrañe3xe7nañ4oeapañañg7we3ñz4ñ0x7iaxñrañb4eaña7w274ñ406

'un dia una liebre orgullosa y veloz vio como una tortuga caminaba por el camino y se le acerco la liebre empezo a burlarse de la lentitud del otro animal y de la longitud de sus patas sin embargo la tortuga le respondio fue estaba segura de fue a pesar de la gran velocidad de la liebre era capaz de ganarla en una carrera la liebre segura de su victoria y considerando el reto imposible de perder acepto ambos pidieron a la zorra fue señalara la meta a lo fue esta acepto al igual fue al cuervo para fue xiciera de juez al llegar el dia de la competicion al empezar la carrera la liebre y la tortuga salieron al mismo tiempo la tortuga avanzaba sin detenerse pero lentamente la liebre era muy veloz y viendo fue sacaba una gran ventaja a la tortuga decidio ir parandose y descansando de vez en cuando pero en una de las ocasiones la liebre se fuedo dormida la tortuga poco a poco siguio avanzando cuando la liebre desperto se encontro con fue la tortuga estaba a punto de cruzar la meta aunfue ecxo